In [1]:
import cv2 
import numpy as np
import tqdm

In [2]:
import os
import glob

In [3]:
files=glob.glob('./tcdata/tile_round1_train_20201231/train_imgs/*.jpg')

### for train

In [7]:
def get_sliced_img_and_init_anno(img_file_path):
    #parameters
    pic_save_path='./tcdata/train_imgs_sliced_all_320/images/'
    anno_save_path='./tcdata/train_imgs_sliced_all_320/labels/'

    target_size=320
    
    try:
        im = cv2.imread(img_file_path)
    except:
        return
    fname=img_file_path.split('\\')[1].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape
    gap_y=hy%target_size//2
    gap_x=wx%target_size//2
    
    num_wx_parts=wx//target_size
    num_hy_parts=hy//target_size
    
    xs=[x for x in range(gap_x,wx,target_size)][:-1]
    ys=[y for y in range(gap_y,hy,target_size)][:-1]
    
    for new_x0 in xs:
        for new_y0 in ys:
            #new boundary
            new_x1=new_x0+target_size
            new_y1=new_y0+target_size
            new_img=im[new_y0:new_y1,new_x0:new_x1,:]
            
#             if hy>=new_y1 and wx>=new_x1:
#                 new_img=im[new_y0:new_y1,new_x0:new_x1,:]
#             else:
#                 new_img=im[new_y0:hy,new_x0:wx,:]
#                 new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
            
            pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
            cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
            
            anno_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.txt'
            open(anno_save_path+anno_fname, "w")
#             with open(anno_save_path+anno_fname,'w') as f:
#                 f.write(out_string)
    

In [9]:
for file_path in files:
    get_sliced_img_and_init_anno(file_path)

In [15]:
file_path

'./tile_round1_train_20201231/train_imgs\\254_99_t20201130153406261_CAM1.jpg'

In [16]:
files[-1]

'./tile_round1_train_20201231/train_imgs\\254_99_t20201130153406261_CAM1.jpg'

### for train val

In [3]:
import json
with open('./tcdata/tile_round1_train_20201231/train_annos.json', 'r') as fp:
     results=json.load(fp)

In [5]:
def get_sliced_yolo_box(dict_data):
    #parameters
    pic_save_path='./tcdata/train_imgs_sliced_all_640_val/images/'
    anno_save_path='./tcdata/train_imgs_sliced_all_640_val/labels/'
    target_size=320
    expand_ratio=2 #2.24
    
    im = cv2.imread('./tcdata/tile_round1_train_20201231/train_imgs/'+dict_data['name'])
    fname=dict_data['name'].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape
    gap_y=hy%target_size//2
    gap_x=wx%target_size//2

    tag_xy=dict_data['bbox']

    new_x0=((round(tag_xy[0])-gap_x)//target_size)*target_size+gap_x
    new_y0=((round(tag_xy[1])-gap_y)//target_size)*target_size+gap_y

    #new img size
    w=round(target_size*expand_ratio)
    h=round(target_size*expand_ratio)
    
    #new boundary
    new_x1=new_x0+w
    new_y1=new_y0+h


    out_x=(np.mean([tag_xy[0],min(tag_xy[2],new_x1)])-new_x0)/w
    out_y=(np.mean([tag_xy[1],min(tag_xy[3],new_y1)])-new_y0)/h
    out_w=(min(tag_xy[2],new_x1)-tag_xy[0])/w
    out_w=min(1,out_w)
    out_h=(min(tag_xy[3],new_y1)-tag_xy[1])/h
    out_h=min(1,out_h)
    
    out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(dict_data['category'],out_x,out_y,out_w,out_h)
    
    
    new_img=im[new_y0:new_y1,new_x0:new_x1,:]
    
#     if hy>=new_y1 and wx>=new_x1:
#         new_img=im[new_y0:new_y1,new_x0:new_x1,:]
#     else:
#         new_img=im[new_y0:hy,new_x0:wx,:]
#         new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)

    pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
    if os.path.exists(pic_fname):
        pass
    else:
        cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

    anno_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.txt'
    with open(anno_save_path+anno_fname,'a+') as f:
        f.write(out_string)

In [ ]:
def get_sliced_yolo_box_nogap(dict_data):
    #parameters
    pic_save_path='./tcdata/train_imgs_sliced_all_1280_val/images/'
    anno_save_path='./tcdata/train_imgs_sliced_all_1280_val/labels/'
    
    if not os.path.exists(pic_save_path): 
        os.makedirs(pic_save_path)
        
    if not os.path.exists(anno_save_path): 
        os.makedirs(anno_save_path)
    
    target_size=1280 #100,180,260
#     expand_ratio=320/target_size #2.24
    if dict_data['category'] in [3,4,5]:
        wh_resize_ratio=1.2
    else:
        wh_resize_ratio=1.05
    
    im = cv2.imread('./tcdata/tile_round1_train_20201231/train_imgs/'+dict_data['name'])
    fname=dict_data['name'].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape

    tag_xy=dict_data['bbox']

    new_x0=(round(tag_xy[0])//target_size)*target_size
    new_y0=(round(tag_xy[1])//target_size)*target_size

    #new img size
#     w=round(target_size*expand_ratio)
#     h=round(target_size*expand_ratio)
    w,h=[1280,1280]
    
    #new boundary
    new_x1=new_x0+w
    new_y1=new_y0+h


    out_x=(np.mean([tag_xy[0],min(tag_xy[2],new_x1)])-new_x0)/w
    out_y=(np.mean([tag_xy[1],min(tag_xy[3],new_y1)])-new_y0)/h
    out_w=(min(tag_xy[2],new_x1)-tag_xy[0])/w
    out_w=min(1,out_w*wh_resize_ratio)
    out_h=(min(tag_xy[3],new_y1)-tag_xy[1])/h
    out_h=min(1,out_h*wh_resize_ratio)
    
    out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(dict_data['category'],out_x,out_y,out_w,out_h)
    
    
    new_img=im[new_y0:new_y1,new_x0:new_x1,:]
    if hy>=new_y1 and wx>=new_x1:
        new_img=im[new_y0:new_y1,new_x0:new_x1,:]
    elif hy>=new_y1 and wx<new_x1:
        new_img=im[new_y0:new_y1,new_x0:wx,:]
        new_img=np.pad(new_img, ((0,0),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
    elif hy<new_y1 and wx>=new_x1:
        new_img=im[new_y0:hy,new_x0:new_x1,:]
        new_img=np.pad(new_img, ((0,new_y1-hy),(0,0),(0,0)), 'constant', constant_values=0)
    else:
        new_img=im[new_y0:hy,new_x0:wx,:]
        new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
#     if hy>=new_y1 and wx>=new_x1:
#         new_img=im[new_y0:new_y1,new_x0:new_x1,:]
#     else:
#         new_img=im[new_y0:hy,new_x0:wx,:]
#         new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)

    pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
    if os.path.exists(pic_fname):
        pass
    else:
        cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

    anno_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.txt'
    with open(anno_save_path+anno_fname,'a+') as f:
        f.write(out_string)

In [6]:
for dict_data in tqdm.tqdm(results):
    get_sliced_yolo_box(dict_data)

100%|██████████████████████████████████████████████████████████████████████████| 15230/15230 [1:27:43<00:00,  2.89it/s]


In [1]:
#### try multiprocessing
%run 04_multiproc_try_cut_target_pic.py

start
15230
all done


In [5]:
type(results[0]['category'])

int

### for predict

In [7]:
files=glob.glob('./tile_round1_testA_20201231/testA_imgs/*.jpg')

In [3]:
files=glob.glob('./tile_round1_testB_20210128/testB_imgs/*.jpg')

In [4]:
files[:10]

['./tile_round1_testB_20210128/testB_imgs\\258_142_t20201202094108644_CAM3.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_142_t20201202094109151_CAM1.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_142_t20201202094109304_CAM2.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_143_t20201202094130288_CAM3.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_143_t20201202094130794_CAM1.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_143_t20201202094130955_CAM2.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_144_t20201202094151605_CAM1.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_144_t20201202094151757_CAM2.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_144_t2020120209415198_CAM3.jpg',
 './tile_round1_testB_20210128/testB_imgs\\258_145_t20201202094210920_CAM3.jpg']

In [5]:
len(files)

1855

In [6]:
files[0].split('\\')[1].split('.j')[0]

'258_142_t20201202094108644_CAM3'

In [7]:
def get_sliced_img(img_file_path):
    #parameters
    pic_save_path='./round1_testA_sliced_all_640/'
    target_size=640
    
    im = cv2.imread(img_file_path)
    fname=img_file_path.split('\\')[1].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape
    gap_y=hy%target_size//2
    gap_x=wx%target_size//2
    
    num_wx_parts=wx//target_size
    num_hy_parts=hy//target_size
    
    xs=[x for x in range(gap_x,wx,target_size)][:-1]
    ys=[y for y in range(gap_y,hy,target_size)][:-1]
    
    for new_x0 in xs:
        for new_y0 in ys:
            #new boundary
            new_x1=new_x0+target_size
            new_y1=new_y0+target_size
            new_img=im[new_y0:new_y1,new_x0:new_x1,:]

#             if hy>=new_y1 and wx>=new_x1:
#                 new_img=im[new_y0:new_y1,new_x0:new_x1,:]
#             elif hy>=new_y1 and wx<new_x1:
#                 new_img=im[new_y0:new_y1,new_x0:wx,:]
#                 new_img=np.pad(new_img, ((0,0),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
#             elif hy<new_y1 and wx>=new_x1:
#                 new_img=im[new_y0:hy,new_x0:new_x1,:]
#                 new_img=np.pad(new_img, ((0,new_y1-hy),(0,0),(0,0)), 'constant', constant_values=0)
#             else:
#                 new_img=im[new_y0:hy,new_x0:wx,:]
#                 new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
                
            pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
            cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
    

In [7]:
def get_sliced_img_nogap(img_file_path):
    #parameters
    target_size=1280
    pic_save_path='./round1_testB_sliced_all_{}/'.format(target_size)
        
    if not os.path.exists(pic_save_path): 
        os.makedirs(pic_save_path)
        
#     if not os.path.exists(anno_save_path): 
#         os.makedirs(anno_save_path)
    
    
    im = cv2.imread(img_file_path)
    fname=img_file_path.split('\\')[1].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape
    
    num_wx_parts=wx//target_size
    num_hy_parts=hy//target_size
    
    xs=[x for x in range(0,wx,target_size)]
    ys=[y for y in range(0,hy,target_size)]
    
    for new_x0 in xs:
        for new_y0 in ys:
            #new boundary
            new_x1=new_x0+target_size
            new_y1=new_y0+target_size
#             new_img=im[new_y0:new_y1,new_x0:new_x1,:]
            
            if hy>=new_y1 and wx>=new_x1:
                new_img=im[new_y0:new_y1,new_x0:new_x1,:]
            elif hy>=new_y1 and wx<new_x1:
                new_img=im[new_y0:new_y1,new_x0:wx,:]
                new_img=np.pad(new_img, ((0,0),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
            elif hy<new_y1 and wx>=new_x1:
                new_img=im[new_y0:hy,new_x0:new_x1,:]
                new_img=np.pad(new_img, ((0,new_y1-hy),(0,0),(0,0)), 'constant', constant_values=0)
            else:
                new_img=im[new_y0:hy,new_x0:wx,:]
                new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)
            
            
            pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
            cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])
    

In [8]:
for file_path in tqdm.tqdm(files):
    get_sliced_img_nogap(file_path)

100%|██████████████████████████████████████████████████████████████████████████████| 1855/1855 [27:26<00:00,  1.13it/s]


In [2]:
file_path

NameError: name 'file_path' is not defined

In [11]:
files[-1]

'./tile_round1_testA_20201231/testA_imgs\\254_96_t20201130153259674_CAM3.jpg'